In [1]:
!pip install -r '/Users/I551982/Desktop/Github/enablement-session-02/requirements.txt'

  Using cached hana_ml-2.22.24110601-py3-none-any.whl.metadata (13 kB)
  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached hdbcli-2.22.32-cp38-abi3-macosx_11_0_arm64.whl.metadata (6.0 kB)
  Using cached ai_core_sdk-2.4.12-py3-none-any.whl.metadata (24 kB)
  Using cached ai_api_client_sdk-2.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached gradio-5.5.0-py3-none-any.whl.metadata (16 kB)
  Using cached gradio_client-1.4.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached langchain-0.2.17-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.2.17-py3-none-any.whl.metadata (2.7 kB)
  Using cached generative_ai_hub_sdk-3.8.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached pydotplus-2.0.2-py3-none-any.whl
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached htmlmin-0.1.12-py3-none-any.whl
  Using cached tqdm-4.67.0-p

In [2]:
import pandas as pd
import json 
import os 
import hana_ml
from hana_ml import ConnectionContext

print(pd.__version__)
print(hana_ml.__version__)

2.2.3
2.22.24110601


### Connect HANA Table


In [3]:
# Connect HANA Table 
with open(os.path.join(os.getcwd(), '/Users/I551982/Desktop/Github/enablement-session-02/config/cesco-poc-hc-service-key.json')) as f:
    hana_env_c = json.load(f)
    port_c  = hana_env_c['port']
    user_c  = hana_env_c['user']
    url_c  = hana_env_c['url']
    pwd_c  = hana_env_c['pwd']

cc = ConnectionContext(address=url_c, port=port_c, user= user_c, password = pwd_c, encrypt=True)
cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA SUMIN""") ## Change SCHEMA to your schema name

#Check hana_ml version 
print(cc.hana_version())
#Check current schema 
print(cc.get_current_schema())

4.00.000.00.1730197503 (fa/CE2024.14)
SUMIN


In [4]:
cc.get_tables()

,TABLE_NAME
0,DEMO_BLOCK_3
1,CATALOG_UPDATED_DEV_SAP_SIERRA


In [4]:
print(cc.get_tables())

                       TABLE_NAME
0  CATALOG_UPDATED_DEV_SAP_SIERRA


### Load dataset

In [5]:
# data = pd.read_csv('/Users/I551982/Desktop/Github/enablement-session-02/data/block_sample_kr_v3.1.csv', index_col=False)
data = pd.read_csv('/Users/I551982/Desktop/Github/enablement-session-02/data/block_sample_kr.csv', index_col=False)

data['BlockID'] = range(1, len(data) + 1)
print(data.columns)

Index(['MaterialNumber', 'MaterialName', 'PurchasingGroupDescription',
       'PurchasingGroup', 'MaterialDescription', 'Total_stock',
       'Unrestricted_use_stock', 'Blocked_stock', 'Block_reason',
       'Blocked_date', 'Solution_date', 'Solution',
       'PurchasingGroupDescription_KR', 'MaterialDescription_KR',
       'Solution_KR', 'Block_reason_KR', 'BlockID'],
      dtype='object')


In [6]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8020 entries, 0 to 8019
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   MaterialNumber                 8020 non-null   int64 
 1   MaterialName                   8020 non-null   object
 2   PurchasingGroupDescription     8020 non-null   object
 3   PurchasingGroup                8020 non-null   object
 4   MaterialDescription            8020 non-null   object
 5   Total_stock                    8020 non-null   int64 
 6   Unrestricted_use_stock         8020 non-null   int64 
 7   Blocked_stock                  8020 non-null   int64 
 8   Block_reason                   8020 non-null   object
 9   Blocked_date                   8020 non-null   object
 10  Solution_date                  8020 non-null   object
 11  Solution                       8020 non-null   object
 12  PurchasingGroupDescription_KR  8020 non-null   object
 13  Mat

### Column re-arrange


In [7]:
data1 = data[['BlockID','MaterialNumber', 'MaterialName', 'MaterialDescription', 'MaterialDescription_KR',
              'PurchasingGroup','PurchasingGroupDescription','PurchasingGroupDescription_KR',
              'Total_stock','Unrestricted_use_stock', 'Blocked_stock',
              'Blocked_date', 'Block_reason','Block_reason_KR',
              'Solution', 'Solution_KR', 'Solution_date' ]]

In [8]:
data1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8020 entries, 0 to 8019
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   BlockID                        8020 non-null   int64 
 1   MaterialNumber                 8020 non-null   int64 
 2   MaterialName                   8020 non-null   object
 3   MaterialDescription            8020 non-null   object
 4   MaterialDescription_KR         7996 non-null   object
 5   PurchasingGroup                8020 non-null   object
 6   PurchasingGroupDescription     8020 non-null   object
 7   PurchasingGroupDescription_KR  8020 non-null   object
 8   Total_stock                    8020 non-null   int64 
 9   Unrestricted_use_stock         8020 non-null   int64 
 10  Blocked_stock                  8020 non-null   int64 
 11  Blocked_date                   8020 non-null   object
 12  Block_reason                   8020 non-null   object
 13  Blo

In [9]:
data.head()

,MaterialNumber,MaterialName,PurchasingGroupDescription,PurchasingGroup,MaterialDescription,Total_stock,Unrestricted_use_stock,Blocked_stock,Block_reason,Blocked_date,Solution_date,Solution,PurchasingGroupDescription_KR,MaterialDescription_KR,Solution_KR,Block_reason_KR,BlockID
0,368814,NORDVIKEN,Bar furniture,P01,"Bar table, 140x80 cm",48663,107,48556,Identification of packaging defects that compr...,3/1/24,4/28/24,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",바 가구,"바 테이블, 140x80 cm","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 조치가 필요...,1
1,368814,NORDVIKEN,Bar furniture,P01,"Bar table, 140x80 cm",48663,107,48556,Identification of packaging defects that compr...,3/1/24,4/28/24,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",바 가구,"바 테이블, 140x80 cm","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",출하 전에 제품의 무결성을 손상시키는 포장 결함을 식별하여 즉각적인 수정 조치가 필...,2
2,368814,NORDVIKEN,Bar furniture,P01,"Bar table, 140x80 cm",48663,107,48556,Identification of packaging defects that compr...,3/1/24,4/28/24,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",바 가구,"바 테이블, 140x80 cm","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 조치가 필요...,3
3,368814,NORDVIKEN,Bar furniture,P01,"Bar table, 140x80 cm",48663,107,48556,Identification of packaging defects that compr...,3/1/24,4/28/24,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",바 가구,"바 테이블, 140x80 cm","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 조치가 필요...,4
4,368814,NORDVIKEN,Bar furniture,P01,"Bar table, 140x80 cm",48663,107,48556,Identification of packaging defects that compr...,3/1/24,4/28/24,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...",바 가구,"바 테이블, 140x80 cm","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",출하 전에 제품의 무결성을 저해하는 포장 결함을 식별함으로써 즉각적인 수정 조치가 ...,5


### Create Tablem in HANA
- Table Name: DEMO-BLOCK3



In [10]:
cursor = cc.connection.cursor()
sql_command = '''CREATE TABLE "DEMO_BLOCK_3" ( 
    "BlockID" BIGINT, 
    "MaterialNumber" BIGINT,
    "MaterialName" NCLOB MEMORY THRESHOLD 0,
    "MaterialDescription" NCLOB MEMORY THRESHOLD 0,
    "MaterialDescription_KR" NCLOB MEMORY THRESHOLD 0,
    "PurchasingGroup"  NVARCHAR(100), 
    "PurchasingGroupDescription" NVARCHAR(5000), 
    "PurchasingGroupDescription_KR" NVARCHAR(5000), 
    "Total_stock" BIGINT,
    "Unrestricted_use_stock" BIGINT, 
    "Blocked_stock" BIGINT, 
    "Blocked_date" DATE,
    "Block_reason" NCLOB MEMORY THRESHOLD 0,
    "Block_reason_KR" NCLOB MEMORY THRESHOLD 0,
    "Solution" NCLOB MEMORY THRESHOLD 0,
    "Solution_KR" NCLOB MEMORY THRESHOLD 0,
    "Solution_date" DATE)'''
    
cursor.execute(sql_command)
cursor.close()

### Read table


In [11]:
cc.has_schema('SUMIN')
cc.get_tables()

,TABLE_NAME
0,DEMO_BLOCK_3
1,CATALOG_UPDATED_DEV_SAP_SIERRA


In [12]:
cc.sql('''SELECT TOP 10 *  FROM DEMO_BLOCK_3''').collect().head()


,BlockID,MaterialNumber,MaterialName,MaterialDescription,MaterialDescription_KR,PurchasingGroup,PurchasingGroupDescription,PurchasingGroupDescription_KR,Total_stock,Unrestricted_use_stock,Blocked_stock,Blocked_date,Block_reason,Block_reason_KR,Solution,Solution_KR,Solution_date


### Load dataset into HANA Table

In [13]:
data1.columns


Index(['BlockID', 'MaterialNumber', 'MaterialName', 'MaterialDescription',
       'MaterialDescription_KR', 'PurchasingGroup',
       'PurchasingGroupDescription', 'PurchasingGroupDescription_KR',
       'Total_stock', 'Unrestricted_use_stock', 'Blocked_stock',
       'Blocked_date', 'Block_reason', 'Block_reason_KR', 'Solution',
       'Solution_KR', 'Solution_date'],
      dtype='object')

#### Change dtype datetime to upload to HANA

In [12]:
data2 = data1.copy()
data2['Blocked_date'] = pd.to_datetime(data2['Blocked_date'])
data2['Solution_date'] = pd.to_datetime(data2['Solution_date'])

/var/folders/69/b5bztsf96dqc792smc_bsrg00000gn/T/ipykernel_86202/3690285358.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data2['Blocked_date'] = pd.to_datetime(data2['Blocked_date'])
/var/folders/69/b5bztsf96dqc792smc_bsrg00000gn/T/ipykernel_86202/3690285358.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data2['Solution_date'] = pd.to_datetime(data2['Solution_date'])


In [13]:
# check dtypes  
data2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8020 entries, 0 to 8019
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   BlockID                        8020 non-null   int64         
 1   MaterialNumber                 8020 non-null   int64         
 2   MaterialName                   8020 non-null   object        
 3   MaterialDescription            8020 non-null   object        
 4   MaterialDescription_KR         7996 non-null   object        
 5   PurchasingGroup                8020 non-null   object        
 6   PurchasingGroupDescription     8020 non-null   object        
 7   PurchasingGroupDescription_KR  8020 non-null   object        
 8   Total_stock                    8020 non-null   int64         
 9   Unrestricted_use_stock         8020 non-null   int64         
 10  Blocked_stock                  8020 non-null   int64         
 11  Blocked_date     

In [14]:
data2.shape

(8020, 17)

In [18]:
# import dataframe into hana table 
from hana_ml.dataframe import create_dataframe_from_pandas
v_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=data2[:100], #### Upload sample dataset to HANA
    table_name="DEMO_BLOCK_3", 
    allow_bigint=True, 
    append=True,
    force=False)

100%|██████████| 1/1 [00:00<00:00, 22.17it/s]


In [10]:
# Understand the data in this newly created table
# you could join multiple text columns in SQL
hdf = cc.sql('''SELECT TOP 10 "BlockID","MaterialNumber", "MaterialName",
             "Blocked_stock","Block_reason","Blocked_date", "Solution", "Solution_KR","Solution_date" 
             FROM DEMO_BLOCK_3''')
df_abstract = hdf.collect()
df_abstract.head()

,BlockID,MaterialNumber,MaterialName,Blocked_stock,Block_reason,Blocked_date,Solution,Solution_KR,Solution_date
0,1,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28
1,2,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28
2,3,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28
3,4,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28
4,5,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28


### Create context column before embedding

In [15]:
data2['Context'] = 'Block reason:' + data2['Block_reason'] \
                    + '\n Material Number:' + data2['MaterialNumber'].astype(str) \
                    + '\n Material Name:' + data2['MaterialName'] \
                    + '\n Material Description:' + data2['MaterialDescription'] \
                    + '\n Purchasing Group:' + data2['PurchasingGroup'] \
                    + '\n Purchasing Group Description:' + data2['PurchasingGroupDescription']
                    
data2['Context_KR'] = '블럭 원인:' + data2['Block_reason_KR'] \
                    + '\n 자재 번호:' + data2['MaterialNumber'].astype(str) \
                    + '\n 자재명:' + data2['MaterialName'] \
                    + '\n 자재 설명:' + data2['MaterialDescription_KR'] \
                    + '\n 자재 그룹:' + data2['PurchasingGroup'] \
                    + '\n 구매 그룹 설명:' + data2['PurchasingGroupDescription_KR']

In [16]:
data2[['Context','Context_KR']].head()

,Context,Context_KR
0,Block reason:Identification of packaging defec...,블럭 원인:출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 ...
1,Block reason:Identification of packaging defec...,블럭 원인:출하 전에 제품의 무결성을 손상시키는 포장 결함을 식별하여 즉각적인 수정...
2,Block reason:Identification of packaging defec...,블럭 원인:출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 ...
3,Block reason:Identification of packaging defec...,블럭 원인:출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 ...
4,Block reason:Identification of packaging defec...,블럭 원인:출하 전에 제품의 무결성을 저해하는 포장 결함을 식별함으로써 즉각적인 수...


## Embedding

### Add Vector Column to HANA

In [24]:
# add a vector column to your table
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE DEMO_BLOCK_3 ADD (VECTOR_EN REAL_VECTOR(1536))'''
cursor.execute(sql_command)
cursor.close()

#### Get Embedding

In [17]:
# get embedding
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model="d4971d165d84d536") -> str: #canary-dc872f9eef04c31a
    response = embeddings.create(
      deployment_id=model,
      input=input
    )
    return response.data[0].embedding

In [18]:
get_embedding('hello world')

[-0.006714438553899527,
 -0.03919780254364014,
 0.03417709469795227,
 0.02876320295035839,
 -0.024785956367850304,
 -0.042010609060525894,
 -0.030260341241955757,
 0.049299705773591995,
 -0.013943045400083065,
 -0.01766320690512657,
 0.015379693359136581,
 -0.026948489248752594,
 -0.02099018171429634,
 -0.02781047858297825,
 0.00861232541501522,
 0.03565911203622818,
 -0.053624771535396576,
 -0.0023515657521784306,
 0.008839164860546589,
 0.04802940413355827,
 0.03711088374257088,
 -0.009270159527659416,
 -0.008740867488086224,
 0.011402446776628494,
 0.014109394513070583,
 -0.002185216872021556,
 -0.037564560770988464,
 0.04539807140827179,
 0.011228537186980247,
 -0.03962123394012451,
 0.023424921557307243,
 -0.05063049495220184,
 0.01199979055672884,
 -5.354939276003279e-05,
 0.01606021076440811,
 0.006151121109724045,
 0.031999439001083374,
 0.003408258082345128,
 -0.008597203530371189,
 -0.010570704005658627,
 -0.03738309070467949,
 -0.034479547291994095,
 0.04999534413218498,
 0.

### Embedding_EN

In [19]:
# generate embeddings from the text
rows = []
for index, row in data2[:100].iterrows():
    text = row['Context']
    try:
        text_vector = get_embedding(input=text,model="d4971d165d84d536")
        myrow = (str(text_vector), row['BlockID'])
        rows.append(myrow)
    except Exception as e:
        print(e)

#### Import embeddings of Context to HANA


In [20]:
# bulk update
# cc.connection.setautocommit(False)
cursor = cc.connection.cursor()
sql = 'UPDATE DEMO_BLOCK_3 SET VECTOR_EN = TO_REAL_VECTOR(?) WHERE "BlockID" = ?'
try:
    cursor.executemany(sql, rows)
except Exception as e:
    cc.connection.rollback()
    print("An error occurred:", e)
try:
    cc.connection.commit()
finally:
    cursor.close()
cc.connection.setautocommit(True)

### Embedding_KR

In [21]:
# add a vector column to your table
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE DEMO_BLOCK_3 ADD (VECTOR_KR REAL_VECTOR(1536))'''
cursor.execute(sql_command)
cursor.close()

In [24]:
data2['Context_KR'][0]

'블럭 원인:출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 조치가 필요합니다.\n 자재 번호:368814\n 자재명:NORDVIKEN\n 자재 설명:바 테이블, 140x80 cm\n 자재 그룹:P01\n 구매 그룹 설명:바 가구'

In [22]:
#Test Embedding
get_embedding(data2['Context_KR'][0])


[-0.01566549949347973,
 0.011042986996471882,
 0.008326196111738682,
 -0.01140598114579916,
 0.007855437695980072,
 -0.028449727222323418,
 0.005927026737481356,
 -0.03226117417216301,
 -0.0087345652282238,
 -0.06824304908514023,
 0.03178474307060242,
 0.025228148326277733,
 -0.018819019198417664,
 -0.005442088004201651,
 0.05277038738131523,
 0.06951352953910828,
 -0.03881777077913284,
 0.010084453038871288,
 -0.07772628962993622,
 -0.00015437918773386627,
 0.015461315400898457,
 0.020270997658371925,
 0.004721770063042641,
 -0.047189339995384216,
 0.008173057809472084,
 0.04503405839204788,
 0.03255610913038254,
 -0.012682135216891766,
 -0.008541724644601345,
 -0.034053463488817215,
 -0.02504665032029152,
 -0.013714401982724667,
 0.04455762729048729,
 -0.05081929266452789,
 -0.002603354398161173,
 -0.013045130297541618,
 0.006273006089031696,
 -0.011785991489887238,
 -0.0031421747989952564,
 0.019261417910456657,
 -0.04517018049955368,
 0.0164708960801363,
 0.035505443811416626,
 0.0

In [25]:
# generate embeddings from the text
rows = []
for index, row in data2[:100].iterrows():
    text = row['Context_KR']
    # print(text)
    try:
        text_vector = get_embedding(input=text,model="d4971d165d84d536")
        myrow = (str(text_vector), row['BlockID'])
        rows.append(myrow)
    except Exception as e:
        print(e)

In [26]:
# bulk update
# cc.connection.setautocommit(False)
cursor = cc.connection.cursor()
sql = 'UPDATE DEMO_BLOCK_3 SET VECTOR_KR = TO_REAL_VECTOR(?) WHERE "BlockID" = ?'
try:
    cursor.executemany(sql, rows)
except Exception as e:
    cc.connection.rollback()
    print("An error occurred:", e)
try:
    cc.connection.commit()
finally:
    cursor.close()
cc.connection.setautocommit(True)

In [28]:
# Check Updates
cc = ConnectionContext(address=url_c, port=port_c, user= user_c, password = pwd_c, encrypt=True)
cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA SUMIN""")

hdf = cc.sql('''SELECT TOP 5 "BlockID","MaterialNumber", "MaterialName",
             "Blocked_stock","Block_reason","Blocked_date", "Solution", "Solution_KR","Solution_date", "VECTOR_KR"
             FROM DEMO_BLOCK_3''')
df_abstract = hdf.collect() 
df_abstract.head(5)

96


,BlockID,MaterialNumber,MaterialName,Blocked_stock,Block_reason,Blocked_date,Solution,Solution_KR,Solution_date,VECTOR_KR
0,1,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28,"[0, 6, 0, 0, 239, 84, 128, 188, 165, 237, 52, ..."
1,2,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28,"[0, 6, 0, 0, 234, 44, 90, 188, 205, 25, 139, 6..."
2,3,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28,"[0, 6, 0, 0, 130, 168, 106, 188, 156, 246, 78,..."
3,4,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28,"[0, 6, 0, 0, 130, 168, 106, 188, 156, 246, 78,..."
4,5,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28,"[0, 6, 0, 0, 235, 165, 101, 188, 0, 14, 38, 60..."


### Create an Embedding to query the HANA Table

In [29]:
text_vector = get_embedding(input = "의자의 포장 불량 배송에 대한 해결책은 무엇입니까?")

In [30]:
#check vector size
len(text_vector)


1536

In [34]:
with open('/Users/I551982/Desktop/Github/enablement-session-02/demo/text_vector_01_KR.txt','w') as output: 
    output.write(str(text_vector))